## Deploy and Discover MCP Server

This notebook demonstrates how to:
1. Deploy an MCP server using the ToolHive operator
2. Connect it to the registry so it's discovered as a `remote` server
3. Verify the server appears in the "default" kubernetes registry
4. Test the server by connecting it to Cursor IDE and running queries.

### Prerequisites

- The MCP Registry must be deployed and accessible (from notebook `02-deploy-registry.ipynb`)
- Port-forward the registry API service: `oc port-forward svc/demo-registry-api 8888:8080`


In [ ]:
import sys
from pathlib import Path

# Add lib directory to Python path to import registry_client
# Notebooks are in notebooks/ directory, so go up one level to project root
sys.path.insert(0, str(Path().absolute().parent / "lib"))

from registry_client import RegistryClient, print_server_details

# Initialize the registry client
client = RegistryClient(host="localhost", port=8888)


### Deploy the Kubernetes MCP Server

The manifest includes annotations that connect the server to the registry:

- `toolhive.stacklok.dev/registry-export: "true"` - Enables registry export
- `toolhive.stacklok.dev/registry-url` - Points to the registry API endpoint
- `toolhive.stacklok.dev/registry-description` - Description shown in the registry.

In [19]:
# Deploy the Kubernetes MCP Server
!oc apply -f ../manifests/kubernetes-mcp-server.yaml


mcpserver.toolhive.stacklok.dev/io-github-stacklok-kubernetes-mcp-server unchanged


In [20]:
# Wait for the server to be ready
!oc wait pod io-github-stacklok-kubernetes-mcp-server-0 --for=condition=Ready


pod/io-github-stacklok-kubernetes-mcp-server-0 condition met


In [21]:
# Verify the server pod is running
!oc get pods -n toolhive-system -l app=io-github-stacklok-kubernetes-mcp-server 
!oc get pods -n toolhive-system -l app.kubernetes.io/name=mcpserver


NAME                                         READY   STATUS    RESTARTS   AGE
io-github-stacklok-kubernetes-mcp-server-0   1/1     Running   0          10m
NAME                                                        READY   STATUS    RESTARTS   AGE
io-github-stacklok-kubernetes-mcp-server-859445785b-rnk87   1/1     Running   0          10m


### Verify Server Discovery in Registry

Check that the server appears in the "default" kubernetes registry as a remote server.


In [ ]:
# Query the default registry for servers
servers = client.get_mcp_servers("default")
print(f"Found {servers.get('metadata', {}).get('count', 0)} server(s) in default registry\n")

for server_item in servers.get("servers", []):
    print("=" * 80)
    print_server_details(server_item)


Found 1 server(s) in default registry

Name:       com.toolhive.k8s.toolhive-system/io-github-stacklok-kubernetes-mcp-server
Description: yet another MCP server
Version:    1.0.0
Remotes:    1 remote endpoint(s)
  - streamable-http: http://demo-registry-api.toolhive-system.svc.cluster.local:8080
Transport:  sse
Identifier: quay.io/containers/kubernetes_mcp_server:latest-linux-amd64



### Port-Forward the Proxy Service

To connect the server to Cursor IDE, we need to expose the proxy service locally.


**Note**: Run the following command in a separate terminal to port-forward the proxy service:

```bash
oc port-forward svc/mcp-io-github-stacklok-kubernetes-mcp-server-proxy -n toolhive-system 9999:8080
```

This will expose the proxy service on `localhost:9999`.


### Connect to Cursor IDE

1. Open Cursor IDE settings
2. Navigate to **Features** → **Model Context Protocol**
3. Click **Add Server** or edit your MCP settings
4. Add a new server configuration:

```json
{
  "mcpServers": {
    "kubernetes": {
      "url": "http://localhost:9999/sse",
      "transport": "sse"
    }
  }
}
```

5. Save the configuration and restart Cursor IDE if needed
6. The Kubernetes MCP server should now be available in your AI agent


### Test the Server

Once connected to Cursor IDE, you can test the server by asking the AI agent to:

**Example queries:**
- "List all namespaces in the cluster"
- "Show me pods in the toolhive-system namespace"
- "What resources are available in the current namespace?"
- "Describe the demo-registry-api pod"

The AI agent will use the Kubernetes MCP server to query your OpenShift cluster and return the results.

**Note**: Depending on the configured RBAC roles, some cluster-wide requests might fail because the deployed MCP server doesn't have the required privileges.


## Conclusion

We successfully:

- ✅ Deployed the Kubernetes MCP Server using the ToolHive operator
- ✅ Connected it to the registry with proper annotations
- ✅ Verified it appears as a remote server in the "default" kubernetes registry
- ✅ Set up port-forwarding for local access
- ✅ Configured Cursor IDE to connect to the server

The Kubernetes MCP server is now ready to use with your AI agent in Cursor IDE. You can query Kubernetes resources, manage deployments, and interact with your OpenShift cluster through natural language queries.
